<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataMinig/blob/main/20231101/%E5%9F%BA%E7%A4%8E%E9%9B%86%E8%A8%88%E3%81%A8%E5%8F%AF%E8%A6%96%E5%8C%96%E3%81%AE%E6%BC%94%E7%BF%92%E3%81%AE%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データマイニング第6回（2023/11/1）

#本日の講義の目的





前回までの講義内容を復習し、基礎集計と可視化を実施してみる

## 準備

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns



---



---



# Booksデータベース

## データの読み込み
* データをダウンロード

    以下のアドレスより、book.csv をPCにダウンロードする。

    https://github.com/yajima-yasutoshi/DataMinig/tree/main/20231101
[リンクテキスト](https://)
* Google Drive のマウントの確認

* データ（ books.csv ）を Google Drive の適当なフォルダーにアップロード

Booksデータベースには、世界各国で

データの説明

項目                  | 説明
--                      | --
 bookID             |  本のID番号
 title              |  本のタイトル
 authors            |  本の著者
 average_rating     |  平均評価
 isbn               |  ISBN番号
 isbn13             |  ISBN番号
 language_code      |  言語
 num_pages          |  ページ数
 ratings_count      |  評価の数
 text_reviews_count |  レビュー文章が記載された数
 publication_date   |  出版日  
 publisher          |  出版社

In [ ]:
# データフレームへの読み込み
file_name = '/content/drive/MyDrive/周南公立大学/講義/データマイニング/データ/books.csv'
# 一部に読み込めないレコードがあるので、error_bad_lines=False を指定している
df = pd.read_csv(file_name, error_bad_lines=False)
df.rename(columns={'  num_pages': 'num_pages'}, inplace=True)

## データの確認

In [ ]:
df.head()

データのレコードの数と列の個数を調べる。

In [ ]:
df.shape

列の型を調と欠損値の有無を確認する

In [ ]:
df.info()

上の結果から、それぞれの項目変数の型、また欠損値が無いことが分かる。

### 型の見分け方

型 | Dtype
-- | --
数値型 | int??, float??,
カテゴリ型 | object


データに登録されている本の平均評価（average_rating）の中央値は何点か。

データに登録されている本のページ数（num_pages）の平均値は何ページか。


## 各項目の調査



*   数値型：最大、最小、平均、中央値など
*   カテゴリ型：カテゴリ数、最頻値など



In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

## データの理解

最も多く本が登録されている著者は誰か

In [ ]:
df['authors'].mode()

In [ ]:
df['authors'].value_counts().sort_values(ascending=False).head(4)

ページ数の分布

In [ ]:
sns.histplot(df['num_pages'], bins=50)

最もページ数（num_pages）の最も多い本は何か。

In [ ]:
df.sort_values(by = 'num_pages', ascending=False).head(3)

average_ratings の分布


In [ ]:
sns.histplot(df['average_rating']) #, binwidth=0.1)

著者の中で、ページ数（num_pages）の平均が最も多い著者は誰か？

In [ ]:
df.groupby('authors')['num_pages'].mean().sort_values(ascending=False)

booksデータベースで、language_code が spa の本は何冊あるか。

In [ ]:
df['language_code'].value_counts()

In [ ]:
df[ df['language_code'] == 'spa' ].shape

出版社 （publisher）の中で、ページ数の平均が最も多い出版社はどれか。

In [ ]:
df.groupby('publisher')['num_pages'].mean().sort_values(ascending=False)
# Group the data by publisher and compute the average number of pages for each publisher
# average_pages_per_publisher = df.groupby('publisher')['num_pages'].mean().reset_index()
# average_pages_per_publisher

出版社（publisher） 毎に ratings_count の合計を計算する。このとき、合計が最も多い publisher はどれか。

In [ ]:
df.groupby('publisher')['ratings_count'].sum().sort_values(ascending=False)

ratings_count の合計の多い出版社 Top5を可視化してみる。

In [ ]:
ratings_count_per_publisher_top5 = df.groupby('publisher')['ratings_count'].sum().sort_values(ascending=False).head(5).reset_index()

In [ ]:
ratings_count_per_publisher_top5

In [ ]:
sns.barplot(x='ratings_count', y='publisher', data=ratings_count_per_publisher_top5)
plt.title('レーティング数の合計')
plt.xlabel('回数')
plt.ylabel('出版社')

language_code が jpn の本に限定した場合、ページ数の平均が最も多い出版社（publisher）はどれか。

In [ ]:
df[ df['language_code'] == 'jpn' ].groupby('publisher')['num_pages'].mean().sort_values(ascending=False)
#df[ df['language_code'] == 'jpn' ].groupby('publisher')['num_pages'].agg(['count', 'mean'])


---
---

# Titanic データの説明

1912年に起きたTitanic号の沈没事故に関するデータを使い、今までの講義内容を復習する。

今回利用するデータは seaborn パッケージに含まれているサンプルデータである。

In [ ]:
# データの読み込み
df = sns.load_dataset('titanic')
df.info()


今回利用するデータの概要は以下の通り。

項目 | 説明
-- | --
survived | 生存したかどうか（0 = No, 1 = Yes）
pclass   | 乗客のクラス（1 = 1st, 2 = 2nd, 3 = 3rd）
sex      | 性別（male, female）
age      | 年齢
sibsp  | タイタニック号に乗っていた兄弟姉妹/配偶者の数
parch | タイタニック号に乗っていた親/子供の数
fare | 乗船料金
embarked |  乗船した港（C = Cherbourg, Q = Queenstown, S = Southampton）
class | 乗客のクラスを文字列で（'First', 'Second', 'Third'）
who | 乗客のタイプ（'man', 'woman', 'child'）
adult_male |  成人男性かどうか（True, False）
deck |  乗客がいたデッキ（A, B, C, D, E, F, G, NaN）
embark_town |  乗船した町の名前（'Southampton', 'Cherbourg', 'Queenstown', NaN）
alive |  生存したかどうかを文字列で入力（'yes', 'no'）
alone | 一人で乗船したかどうか（True, False）

なお、欠損値: age、embarked などの特定の列に欠損値が存在する。


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
# データの前処理
df['age'].fillna(df['age'].median(), inplace=True)  # 'age'列の欠損値を中央値で埋める
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)  # 'embarked'列の欠損値を最頻値で埋める

In [ ]:
# 基礎集計
survival_count = df['survived'].value_counts()  # 生存者と死亡者の数を集計
survival_rate_by_gender = df.groupby('sex')['survived'].mean()  # 性別ごとの生存率を集計

In [ ]:
# データの可視化（Seaborn使用）
sns.countplot(x='survived', data=df)  # 生存者と死亡者の数を棒グラフで表示
plt.show()

sns.barplot(x='sex', y='survived', data=df)  # 性別ごとの生存率を棒グラフで表示
plt.show()

sns.boxplot(x='survived', y='age', data=df)  # 年齢と生存率の関係を箱ひげ図で表示
plt.show()


In [ ]:
sns.countplot(x='survived', data=df)  # 生存者と死亡者の数を棒グラフで表示

In [ ]:
sns.barplot(x='survived', data=df)